# 2 Variables

$Y = A \oplus B$

In [5]:
import pandas as pd

TRUTH_TABLE = pd.DataFrame([
    [str(x) for x in [0,0,0]],
    [str(x) for x in [0,1,1]],
    [str(x) for x in [1,0,1]],
    [str(x) for x in [1,1,0]],
])

TRUTH_TABLE.columns = ['A','B','Y']
TRUTH_TABLE

,A,B,Y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [7]:
def gen_2_kmap(TRUTH_TABLE):
    kmap = [[-1 for _ in range(2)] for _ in range(2)]

    for a in range(2):
        for b in range(2):
            kmap[b][a] = TRUTH_TABLE[(TRUTH_TABLE['A']==str(a)) & (TRUTH_TABLE['B']==str(b))].iloc[0,-1]
    df = pd.DataFrame(kmap)
    df.index = ['0','1']
    df.columns = ['0','1']
    return df

kmap = gen_2_kmap(TRUTH_TABLE)

def determine_state(var, value):
    if value=='1':
        return var
    else:
        return var+"\'"
    
kmap

,0,1
0,0,1
1,1,0


In [ ]:
def gen_state(var, value):
    state = ""
    for v, va in zip(var,value):
        state+=determine_state(v,va)

    return state

def isValidRectangle(kmap, rowspan, colspan, row, col):
    m,n = kmap.shape
    if(rowspan>m or colspan>n):
        return False
    
    for r in range(rowspan):
        for c in range(colspan):
            if kmap.iloc[(row+r)%m,(col+c)%n]=="0":
                return False
    
    return True

def determine_rects(kmap):
    rects = []
    m, n = kmap.shape
    current_rects = set()

    row_spans = [1 << i for i in range(m.bit_length()) if 1 << i <= m]
    col_spans = [1 << j for j in range(n.bit_length()) if 1 << j <= n]
    print(f'{row_spans}\n{col_spans}')
    for rowspan in row_spans:
        for colspan in col_spans:
            for row in range(m):
                for col in range(n):
                    if isValidRectangle(kmap, rowspan, colspan, row, col):
                        pts = tuple(sorted(
                            ((row + i) % m, (col + j) % n)
                            for i in range(rowspan)
                            for j in range(colspan)
                        ))
                        if pts not in current_rects:
                            current_rects.add(pts)
                            rects.append(((rowspan, colspan), (row, col)))
    rects.sort(key=lambda rect: rect[0][0]*rect[0][1], reverse=True)
    return rects

def select_rects(kmap, rects):
    
    pass
def gen_boolean_map(vars, row_order, col_order):
    b_map = [["" for _ in range(len(col_order))] for _ in range(len(row_order))]
    for row_val in row_order:
        for col_val in col_order:
            b_map[row_order.index(row_val)][col_order.index(col_val)] = gen_state(vars,col_val+row_val)
    
    return pd.DataFrame(b_map)

bmap = gen_boolean_map("AB",["0","1"],["0","1"])

In [57]:
def solve_kmap(kmap,bmap):
    m,n = kmap.shape
    expr = ""
    for i in range(m):
        for j in range(n):
            if(kmap.iloc[i,j]=="1"):
                expr += bmap.iloc[i,j] + " + "

    return expr[:-3]

determine_rects(kmap)

[1, 2]
[1, 2]
Checking 1 1 0 1 => ((0, 1),)
Checking 1 1 1 0 => ((1, 0),)


[((1, 1), (0, 1)), ((1, 1), (1, 0))]

# 3 Variables

$Y = A\cdot B + C'$

In [27]:
TRUTH_TABLE_3 = pd.DataFrame([
    [str(x) for x in [0,0,0,1]],
    [str(x) for x in [0,0,1,0]],
    [str(x) for x in [0,1,0,1]],
    [str(x) for x in [0,1,1,0]],
    [str(x) for x in [1,0,0,1]],
    [str(x) for x in [1,0,1,0]],
    [str(x) for x in [1,1,0,1]],
    [str(x) for x in [1,1,1,1]]
])

TRUTH_TABLE_3.columns = ['A','B','C','Y']
TRUTH_TABLE_3

,A,B,C,Y
0,0,0,0,1
1,0,0,1,0
2,0,1,0,1
3,0,1,1,0
4,1,0,0,1
5,1,0,1,0
6,1,1,0,1
7,1,1,1,1


In [68]:
def map2(gray_order,a,b):
    return gray_order.index(str(a)+str(b))

def gen_3_kmap(TRUTH_TABLE):
    kmap = [[0 for _ in range(4)] for _ in range(2)]
    gray_code_order = ['00','01','11','10']
    for a in range(2):
        for b in range(2):
            for c in range(2):
                kmap[c][map2(gray_code_order,a,b)] = TRUTH_TABLE[(TRUTH_TABLE['A']==str(a)) & (TRUTH_TABLE['B']==str(b)) & (TRUTH_TABLE['C']==str(c))].iloc[0,-1]

    kmap_df = pd.DataFrame(kmap,index=['0','1'])
    kmap_df.columns = gray_code_order
    return kmap_df

kmap3 = gen_3_kmap(TRUTH_TABLE_3)
bmap3 = gen_boolean_map("ABC",['0','1'],['00','01','11','10'])
print(kmap3)
determine_rects(kmap3)

  00 01 11 10
0  1  1  1  1
1  0  0  1  0
[1, 2]
[1, 2, 4]


[((1, 4), (0, 0)),
 ((1, 2), (0, 0)),
 ((1, 2), (0, 1)),
 ((1, 2), (0, 2)),
 ((1, 2), (0, 3)),
 ((2, 1), (0, 2)),
 ((1, 1), (0, 0)),
 ((1, 1), (0, 1)),
 ((1, 1), (0, 2)),
 ((1, 1), (0, 3)),
 ((1, 1), (1, 2))]